In [1]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np



In [2]:
# 이동평균계산
class moving_avg(nn.Module):
    """
    Moving average block to highlight the trend of time series
    """
    def __init__(self, kernel_size, stride):
        super(moving_avg, self).__init__()
        self.kernel_size = kernel_size
        self.avg = nn.AvgPool1d(kernel_size=kernel_size, stride=stride, padding=0)
#AvgPoolid 이동평균계산하기 양끝을 패딩함
    def forward(self, x):
        # padding on the both ends of time series
        front = x[:, 0:1, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        end = x[:, -1:, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        x = torch.cat([front, x, end], dim=1)
        x = self.avg(x.permute(0, 2, 1))
        x = x.permute(0, 2, 1)
        return x


In [3]:
# 시계열 분해하기 이동펴ㅕㅇ균(트랜드계산하고 시계열 데이터에 빼서 계절성 추출 
class series_decomp(nn.Module):
    """
    Series decomposition block
    """
    def __init__(self, kernel_size):
        super(series_decomp, self).__init__()
        self.moving_avg = moving_avg(kernel_size, stride=1) #이동평균계산

    def forward(self, x):
        moving_mean = self.moving_avg(x)
        res = x - moving_mean
        return res, moving_mean

In [4]:
#사용자 정의 모듈 주요 클래스 
# 트랜드와 계절성에 대한 복잡한 패턴을 학습 
# 트랜드와 계절성 분리하고 각가에 대한 예측을 수행 
class Model(nn.Module):
    """
    DLinear
    """
    def __init__(self, configs):
        super(Model, self).__init__()
        self.Lag  = configs.Lag
        self.Horizon = configs.Horizon

        # Decompsition Kernel Size
        kernel_size       = configs.kernel_size
        self.decompsition = series_decomp(kernel_size)
        self.individual   = configs.individual
        self.channels     = configs.enc_in

        if self.individual:
            self.Linear_Seasonal = nn.ModuleList()
            self.Linear_Trend    = nn.ModuleList()
            self.Linear_Decoder  = nn.ModuleList()
            for i in range(self.channels):
                self.Linear_Seasonal.append(nn.Linear(self.Lag,self.Horizon))
                self.Linear_Seasonal[i].weight = nn.Parameter((1/self.Lag)*torch.ones([self.Horizon,self.Lag]))
                self.Linear_Trend.append(nn.Linear(self.Lag,self.Horizon))
                self.Linear_Trend[i].weight = nn.Parameter((1/self.Lag)*torch.ones([self.Horizon,self.Lag]))
                self.Linear_Decoder.append(nn.Linear(self.Lag,self.Horizon))
        else:
            self.Linear_Seasonal = nn.Linear(self.Lag,self.Horizon)
            self.Linear_Trend = nn.Linear(self.Lag,self.Horizon)
            self.Linear_Decoder = nn.Linear(self.Lag,self.Horizon)
            self.Linear_Seasonal.weight = nn.Parameter((1/self.Lag)*torch.ones([self.Horizon,self.Lag]))
            self.Linear_Trend.weight = nn.Parameter((1/self.Lag)*torch.ones([self.Horizon,self.Lag]))
#forward는 모델이 입력 뎅디터를 받아 결과를 출력하는 방법임 각각(트랜드 계절성) 에 대해 선형변환을 적용후 다시 합치는 과정 
    def forward(self, x):
        # x: [Batch, Input length, Channel]
        seasonal_init, trend_init = self.decompsition(x)
        seasonal_init, trend_init = seasonal_init.permute(0,2,1), trend_init.permute(0,2,1)
        if self.individual:
            seasonal_output = torch.zeros([seasonal_init.size(0),seasonal_init.size(1),self.Horizon],dtype=seasonal_init.dtype).to(seasonal_init.device)
            trend_output = torch.zeros([trend_init.size(0),trend_init.size(1),self.Horizon],dtype=trend_init.dtype).to(trend_init.device)
            for i in range(self.channels):
                seasonal_output[:,i,:] = self.Linear_Seasonal[i](seasonal_init[:,i,:])
                trend_output[:,i,:] = self.Linear_Trend[i](trend_init[:,i,:])
        else:
            seasonal_output = self.Linear_Seasonal(seasonal_init)
            trend_output = self.Linear_Trend(trend_init)

        x = seasonal_output + trend_output
        return x.permute(0,2,1) # to [Batch, Output length, Channel]

In [5]:
# 데이터 준비
data = pd.read_csv('6_20.csv') # 파일명은 실제 데이터 파일 경로로 수정
data = data[['공급능력(MW)', '현재수요(MW)', '공급예비력(MW)', '공급예비율(퍼센트)', '운영예비력(MW)', '운영예비율(퍼센트)', '요일', 'is_holiday_or_weekend', '요일1', '요일2', '요일3']]
data = torch.Tensor(np.array(data))


In [6]:
len(data)

1135579

In [7]:
# Configurations
class Config:
    def __init__(self):
        self.Lag = 105120 # 1년 데이터 (5분 간격)
        self.Horizon = 72 # 6시간 데이터 (5분 간격)
        self.kernel_size = 3 # kernel size for moving average
        self.individual = False # individual linear for each channel
        self.enc_in = data.size(-1) # number of channels

configs = Config()

def create_sequences(input_data, target_data, lag, horizon):
    input_seq = []
    target_seq = []

    L = len(input_data)
    for i in range(L - lag - horizon + 1):
        input_seq.append(input_data[i:i+lag].tolist())
        target_seq.append(target_data[i+lag:i+lag+horizon].tolist())

    return torch.tensor(input_seq), torch.tensor(target_seq)


input_data = data[:-configs.Horizon] # 입력 데이터
target_data = data[configs.Lag:] # 예측 대상 데이터

input_data, target_data = create_sequences(input_data, target_data, configs.Lag, configs.Horizon)

dataset = TensorDataset(input_data, target_data)
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42) 
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)


# 모델 초기화
model = Model(configs)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)




# 모델 학습
for epoch in range(100): 
    train_loss = 0
    for batch_idx, (data, target) in enumerate(train_dataloader):
        prediction = model(data)
        loss = F.mse_loss(prediction, target) #
        train_loss += loss.item()

        # 역전파 및 가중치 업데이트
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_loss /= len(train_dataloader)
    print('Train Epoch: {}, Avg. Loss: {}'.format(epoch, train_loss))

  
    if epoch % 10 == 0:
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(test_dataloader):
                prediction = model(data)
                loss = F.mse_loss(prediction, target) 
                val_loss += loss.item()

        val_loss /= len(test_dataloader)
        print('Validation Epoch: {}, Avg. Loss: {}'.format(epoch, val_loss))

        # Visualizing model prediction
        sample_data = next(iter(test_dataloader))[0][0] 
        model_output = model(sample_data.unsqueeze(0)) 
        plt.figure(figsize=(12, 4))
        plt.plot(sample_data.numpy(), label='Input')
        plt.plot(range(configs.Lag, configs.Lag + configs.Horizon), model_output[0].detach().numpy(), label='Prediction')
        plt.legend()
        plt.show()

        model.train()

In [ ]:
# # Configurations
# class Config:
#     def __init__(self):
#         self.Lag = 105120 # 1년 데이터 (5분 간격)
#         self.Horizon = 72 # 6시간 데이터 (5분 간격)
#         self.kernel_size = 3 # kernel size for moving average
#         self.individual = False # individual linear for each channel
#         self.enc_in = data.size(-1) # number of channels

# configs = Config()

# def create_sequences(input_data, target_data, lag, horizon):
#     input_seq = []
#     target_seq = []

#     L = len(input_data)
#     for i in range(L - lag - horizon + 1):
#         input_seq.append(input_data[i:i+lag].tolist())
#         target_seq.append(target_data[i+lag:i+lag+horizon].tolist())

#     return torch.tensor(input_seq), torch.tensor(target_seq)


# input_data = data[:-configs.Horizon] # 입력 데이터
# target_data = data[configs.Lag:] # 예측 대상 데이터

# input_data, target_data = create_sequences(input_data, target_data, configs.Lag, configs.Horizon)

# dataset = TensorDataset(input_data, target_data)
# train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42) # 80:20으로 train-test 데이터 분할

# train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)


# # 모델 초기화
# model = Model(configs)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)




# # 모델 학습
# for epoch in range(100): # 100 epochs
#     train_loss = 0
#     for batch_idx, (data, target) in enumerate(train_dataloader):
#         prediction = model(data)
#         loss = F.mse_loss(prediction, target) # Mean Squared Error loss
#         train_loss += loss.item()

#         # 역전파 및 가중치 업데이트
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#     train_loss /= len(train_dataloader)
#     print('Train Epoch: {}, Avg. Loss: {}'.format(epoch, train_loss))
   

#     # Validate and print loss every 10 epochs
#     if epoch % 10 == 0:
#         model.eval()
#         val_loss = 0
#         with torch.no_grad():
#             for batch_idx, (data, target) in enumerate(test_dataloader):
#                 prediction = model(data)
#                 loss = F.mse_loss(prediction, target) # Mean Squared Error loss
#                 val_loss += loss.item()

#         val_loss /= len(test_dataloader)
#         print('Validation Epoch: {}, Avg. Loss: {}'.format(epoch, val_loss))

#         # Visualizing model prediction
#         sample_data = next(iter(test_dataloader))[0][0] # get a sample data from test dataloader
#         model_output = model(sample_data.unsqueeze(0)) # get model output
#         plt.figure(figsize=(12, 4))
#         plt.plot(sample_data.numpy(), label='Input')
#         plt.plot(range(configs.Lag, configs.Lag + configs.Horizon), model_output[0].detach().numpy(), label='Prediction')
#         plt.legend()
#         plt.show()

#         model.train()

In [ ]:
print('Train Epoch: {}, Avg. Loss: {}'.format(epoch, train_loss))